# 贝叶斯定理及统计计算

## 历时诠释
根据数据集 $D$ 的内容变化更新假设概率 $H$，这种对贝叶斯定理的理解被称为“历时诠释”。  
$$P(H|D) = \frac{P(H)P(D|H)}{P(D)}$$
- $P(H)$ 称为**先验概率**，即在得到新数据前某一假设的概率。
- $P(H|D)$ 称为**后验概率**，即在看到新数据后，我们要计算的该假设的概率。
- $P(D|H)$ 是该假设下得到这一数据的概率，称为**似然度**。
- $P(D)$ 是在任何假设下得到这一数据的概率，称为**标准化常量**。

最常见的，我们可以指定一组如下的假设集来简化。  
**互斥的**：集合中，至多一个假设为真。  
**完备的**：集合中，至少一个假设必为真，且集合包含了所有的假设。

In [1]:
from thinkbayes import Pmf
pmf = Pmf()
pmf.Set('Bowl1', 0.5)
pmf.Set('Bowl2', 0.5)
pmf.Mult('Bowl1', 0.75)
pmf.Mult('Bowl2', 0.5)
pmf.Normalize()
print(pmf.Prob('Bowl1'))

0.6


## 曲奇饼问题

In [7]:
class Cookie(Pmf):
    def __init__(self, hypos):
        Pmf.__init__(self)
        # 给每个假设赋予相同的先验概率
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
        
    def Update(self, data):
        for hypo in self.Values():
            # 似然度
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()
    
    # 碗名和碗中曲奇饼的比例
    mixes = {'Bowl1':dict(vanilla=0.75, chocolate=0.25),
             'Bowl2':dict(vanilla=0.5, chocolate=0.5)}
    
    def Likelihood(self, data, hypo):
        mix = self.mixes[hypo]
        like = mix[data]
        return like

In [8]:
hypos = ['Bowl1', 'Bowl2']
pmf = Cookie(hypos)
pmf.Update('vanilla')
for hypo, prob in pmf.Items():
    print(hypo, prob)

('Bowl1', 0.6000000000000001)
('Bowl2', 0.4)


## Monty Hall 难题

In [12]:
class Monty(Pmf):
    # 代表一套假设及其概率
    def __init__(self, hypos):
        # 初始化分配
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
        
    def Update(self, data):
        # 跟新基于该数据的每个假设
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()
        
    def Likelihood(self, data, hypo):
        # 计算似然
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1

In [13]:
hypos = 'ABC'
pmf = Monty(hypos)
data = 'B'
pmf.Update(data)
for hypo, prob in pmf.Items():
    print(hypo, prob)

('A', 0.3333333333333333)
('C', 0.6666666666666666)
('B', 0.0)


## 封装框架

In [14]:
from thinkbayes import Suite
class TestMonty(Suite):
    def Likelihood(self, data, hypo):
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1

suite = TestMonty('ABC')
suite.Update('B')
suite.Print()

A 0.333333333333
B 0.0
C 0.666666666667


## M&M豆问题

In [15]:
class M_and_M(Suite):
    mix94 = dict(brown=30, yellow=20, red=20,
                 green=10, orange=10, tan=10)
    mix96 = dict(blue=24, green=20, orange=16,
                 yellow=14, red=13, brown=13)
    hypoA = dict(bag1 = mix94, bag2 = mix96)
    hypoB = dict(bag1 = mix96, bag2 = mix94)
    hypotheses = dict(A=hypoA, B=hypoB)
    def Likelihood(self, data, hypo):
        bag, color = data
        mix = self.hypotheses[hypo][bag]
        like = mix[color]
        return like
    
suite = M_and_M('AB')
suite.Update(('bag1', 'yellow'))
suite.Update(('bag2', 'green'))
suite.Print()

A 0.740740740741
B 0.259259259259
